In [1]:
from pyomo.environ import *
import pandas as pd

In [2]:
# create the model 
model = AbstractModel()

# declare sets
model.I = Param(mutable=True, default=7)
model.H = Param(mutable=True, default=2)
model.C = Param(mutable=True, default=1)
model.R = Param(default=model.C+model.H)
model.S = Param(mutable=True, default=2)

model.i = RangeSet(1, model.I)I
model.j = Set(initialize=model.i)
model.h = RangeSet(1,model.H)
model.c = RangeSet(model.H+1,model.H+model.C)
model.r = model.h | model.c
model.s = RangeSet(1,model.S)

# declare parameters
model.T = Param(model.i, model.r, within=Reals, mutable=True)
#model.Tc = Param(model.i, model.h, model.c, within=Reals, mutable=True)
model.Tc = Param(model.h, model.c, within=Reals, mutable=True)
model.TP = Param(model.i, model.j, within=Binary, mutable=True)
model.PW = Param(model.i, mutable=True)
model.PWC = Param(model.h, mutable=True)
model.bigM = Param(default=250)

# declare variables
model.x = Var(model.i, model.r, model.s, within=Binary)
model.w = Var(model.i, model.j, model.r, within=Binary)
model.z = Var(model.i, model.h, model.c, model.s, within=Binary)
model.y = Var(model.r, model.s, within=Binary)
model.st = Var(model.i, within=NonNegativeReals)
model.ft = Var(model.i, within=NonNegativeReals)
model.ct = Var(within=NonNegativeReals)
model.mtpw = Var(within=NonNegativeReals)

In [3]:
# declare constraints
def C1_rule(model, i, j):
    if value(model.TP[i,j]) == 1:
        return model.st[j] >= model.ft[i]
    else:
        return Constraint.Skip
model.C1 = Constraint(model.i, model.j, rule=C1_rule)

def C2_rule(model, i, j):
    if i != j:
        return model.st[j] >= model.ft[i] - model.bigM*(1-sum(model.w[i,j,r] for r in model.r))
    else:
        return Constraint.Skip
model.C2 = Constraint(model.i, model.j, rule=C2_rule)

def C3a_rule(model, i):
    F1 = sum(model.x[i,r,s]*model.T[i,r] for r in model.r for s in model.s)
    F2 = sum(model.z[i,h,c,s] for h in model.h for c in model.c for s in model.s)
    return model.ft[i] >= model.st[i] + F1 - model.bigM * F2
model.C3a = Constraint(model.i, rule=C3a_rule)

def C3b_rule(model, i):
#    F = sum(model.z[i,h,c,s]*model.Tc[i,h,c] for h in model.h for c in model.c for s in model.s)
    F = sum(model.z[i,h,c,s]*model.Tc[h,c] for h in model.h for c in model.c for s in model.s)
    return model.ft[i] >= model.st[i] + F
model.C3b = Constraint(model.i, rule=C3b_rule)

def C4_rule(model, i, j, s):
    F1 = sum(model.x[i,r,s] for r in model.r)
    F2 = sum(model.z[i,h,c,s] for h in model.h for c in model.c)
    F3 = sum(model.x[j,r,s] for r in model.r)
    F4 = sum(model.z[j,h,c,s] for h in model.h for c in model.c)
    return model.ct >= model.ft[i] - model.st[j] - model.bigM * (1 - (F1 - F2)) - model.bigM * (1 - (F3 - F4)) 
model.C4 = Constraint(model.i, model.j, model.s, rule=C4_rule)

def C5_rule(model, s):
    F1 = sum(model.x[i,h,s]*model.PW[i] for i in model.i for h in model.h)
    F2 = sum(model.z[i,h,c,s]*model.PW[i] for i in model.i for h in model.h for c in model.c)
    return F1 - F2 <= model.mtpw
#model.C5 = Constraint(model.s, rule=C5_rule)

def C6_rule(model, i):
    F1 = sum(model.x[i,r,s] for r in model.r for s in model.s)
    F2 = sum(model.z[i,h,c,s] for h in model.h for c in model.c for s in model.s) 
    return F1 - F2 == 1 
model.C6 = Constraint(model.i, rule=C6_rule)

def C61_rule(model, i, h, c, s):
    return model.z[i,h,c,s] <= (model.x[i,h,s]+model.x[i,c,s])/2
model.C61 = Constraint(model.i, model.h, model.c, model.s, rule=C61_rule)

def C7_rule(model, i, r, s):
    return model.x[i,r,s] <= model.y[r,s]
model.C7 = Constraint(model.i, model.r, model.s, rule=C7_rule)

def C8_rule(model, r):
    return sum(model.y[r,s] for s in model.s) <= 1
model.C8 = Constraint(model.r, rule=C8_rule)

def C9_rule(model, s):
    return sum(model.y[h,s] for h in model.h) == 1
model.C9 = Constraint(model.s, rule=C9_rule)

def C10_rule(model, s):
    return sum(model.y[c,s] for c in model.c) <= 1
model.C10 = Constraint(model.s, rule=C10_rule)

def C11_rule(model, i, j):
    if value(model.TP[i,j]) == 1:
        F1 = sum(model.x[i,r,s]*s for r in model.r for s in model.s)
        F2 = sum(model.z[i,h,c,s]*s for h in model.h for c in model.c for s in model.s)
        F3 = sum(model.x[j,r,s]*s for r in model.r for s in model.s)
        F4 = sum(model.z[j,h,c,s]*s for h in model.h for c in model.c for s in model.s)
        return F1 - F2 <= F3 - F4
    else:
        return Constraint.Skip
model.C11 = Constraint(model.i, model.j, rule=C11_rule)

def C12_rule(model, r):
    F1 = sum(model.w[i,j,r] for i in model.i for j in model.j)
    F2 = sum(model.x[i,r,s] for i in model.i for s in model.s)
    return F1 == F2 - 1
model.C12 = Constraint(model.r, rule=C12_rule)

def C13_rule(model, i, j, r):
    return model.w[i,j,r] <= (sum(model.x[i,r,s] for s in model.s) + sum(model.x[j,r,s] for s in model.s))/2
model.C13 = Constraint(model.i, model.j, model.r, rule=C13_rule)

def C14_rule(model, i, r):
    return model.w[i,i,r] == 0
model.C14 = Constraint(model.i, model.r, rule=C14_rule)

def C15_rule(model, i, j, r):
    return model.w[i,j,r] + model.w[j,i,r] <= 1
model.C15 = Constraint(model.i, model.j, model.r, rule=C15_rule)

def C16_rule(model, i, r):
    return sum(model.w[i,j,r] for j in model.j) <= 1
model.C16 = Constraint(model.i, model.r, rule=C16_rule)

def C17_rule(model, j, r):
    return sum(model.w[i,j,r] for i in model.i) <= 1
model.C17 = Constraint(model.j, model.r, rule=C17_rule)

def C18_rule(model, i, h, s):
    return model.x[i,h,s]*model.PW[i] <= model.PWC[h] 
#model.C18 = Constraint(model.i, model.h, model.s, rule=C18_rule)

In [4]:
# declare objectives
model.obj1 = Objective(expr=model.ct, sense=minimize)
model.obj2 = Objective(expr=model.mtpw, sense=minimize)

# declare solver
opt = SolverFactory('glpk')

In [5]:
model.obj1.activate()
model.obj2.deactivate()

In [6]:
instance = model.create_instance('data-small.dat')
results = opt.solve(instance)

    mutable Param).  The linkage between this RangeSet and the original source
    data will be broken, so updating the data value in the future will not be
    reflected in this RangeSet.  To suppress this warning, explicitly convert
    the source data to a constant type (e.g., float, int, or immutable Param)
    mutable Param).  The linkage between this RangeSet and the original source
    data will be broken, so updating the data value in the future will not be
    reflected in this RangeSet.  To suppress this warning, explicitly convert
    the source data to a constant type (e.g., float, int, or immutable Param)
    mutable Param).  The linkage between this RangeSet and the original source
    data will be broken, so updating the data value in the future will not be
    reflected in this RangeSet.  To suppress this warning, explicitly convert
    the source data to a constant type (e.g., float, int, or immutable Param)
    mutable Param).  The linkage between this RangeSet and th

In [7]:
print(results)


Problem: 
- Name: unknown
  Lower bound: 11.0
  Upper bound: 11.0
  Number of objectives: 1
  Number of constraints: 615
  Number of variables: 239
  Number of nonzeros: 3437
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 100985
      Number of created subproblems: 100985
  Error rc: 0
  Time: 57.91649103164673
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [8]:
value(instance.obj1)

11.0

In [9]:
for s in instance.s:
    print('Station: ',str(s))
    for i in instance.i:
        SH = 0
        SC = 0
        H = []
        C = []
        for h in instance.h:
            if value(instance.x[i,h,s]) == 1:
                SH += 1
                H = h
        for c in instance.c:
            if value(instance.x[i,c,s]) == 1:
                SC += 1
                C = c

        if (SH == 1) and (SC == 0):
            print('task: ',str(i),' with human: ',str(H),' with Task time: ',value(instance.T[i,H]))
        elif (SH == 0) and (SC == 1):
            print('task: ',str(i),' with Cobot: ',str(C),' with Task time: ',value(instance.T[i,C]))
        elif (SH == 1) and (SC == 1):
            print('task: ',str(i),' with human: ',str(H),'and Cobot: ',str(C),' with Task time: ',value(instance.Tc[H,C]))

Station:  1
task:  1  with human:  1  with Task time:  7
Station:  2
task:  2  with human:  2  with Task time:  2
task:  3  with human:  2  with Task time:  6
task:  4  with Cobot:  3  with Task time:  6
task:  5  with Cobot:  3  with Task time:  3
task:  6  with human:  2  with Task time:  1
task:  7  with Cobot:  3  with Task time:  2


In [10]:
for s in instance.s:
    print(value(instance.ct))

11.0
11.0


In [11]:
for i in instance.i:
    print('task: ',str(i),' ST: ',value(instance.st[i]),' and FT: ',value(instance.ft[i]) )

task:  1  ST:  0.0  and FT:  7.0
task:  2  ST:  7.0  and FT:  9.0
task:  3  ST:  9.0  and FT:  15.0
task:  4  ST:  7.0  and FT:  13.0
task:  5  ST:  13.0  and FT:  16.0
task:  6  ST:  15.0  and FT:  16.0
task:  7  ST:  16.0  and FT:  18.0


In [12]:
for i in instance.i:
  for j in instance.j:
    for r in instance.r:
      if value(instance.w[i,j,r]) == 1:
        print('task: ',str(i),' and task:',str(j),'with resource: ',str(r))

task:  2  and task: 3 with resource:  2
task:  3  and task: 6 with resource:  2
task:  4  and task: 5 with resource:  3
task:  5  and task: 7 with resource:  3
